1. *Самостоятельно повторить tfidf (документация https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

In [1]:
import pandas as pd

Наши новости

In [3]:
news = pd.read_csv("articles.csv")
print(news.shape)
news.head(3)

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


Загрузим пользователей и списки последних прочитанных новостей

In [4]:
users = pd.read_csv("users_articles.csv")
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


Итак, нам нужно получить векторные представления пользователей на основе прочитанных ими новостей и самих новостей

### 1. Получаем векторные представления новостей

In [5]:
!pip install razdel pymorphy2

  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=b1422b6b39fc11dde75dbb200f2f95088aa0b6152735f17f87d1ce9ecc72d5e6
  Stored in directory: c:\users\hp\appdata\local\pip\cache\wheels\70\4a\46\1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt


In [6]:
# предобработка текстов
import re
import numpy as np
from gensim.corpora.dictionary import Dictionary
from razdel import tokenize  # сегментация русскоязычного текста на токены и предложения https://github.com/natasha/razdel
import pymorphy2  # Морфологический анализатор

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

stopword_ru = stopwords.words('russian')
print(len(stopword_ru))

!wget 'https://drive.google.com/uc?export=download&id=17wVn5YPpMjHToctGgff_KfSeWcIIlf7c' -O stopwords.txt

In [8]:
with open('stopwords.txt') as f:
    stopword_ru = [w.strip() for w in f.readlines() if w]
    
#stopword_ru += additional_stopwords
len(stopword_ru)

625

In [9]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    text = re.sub('n', ' ', text)
    
    return text

cache = {}
morph = pymorphy2.MorphAnalyzer()

def lemmatization(text):    
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист лемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w) > 1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords = [i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [10]:
from tqdm import tqdm
tqdm.pandas()

# Запускаем очистку текста. Будет долго...
news['title'] = news['title'].progress_apply(lambda x: clean_text(x))

  0%|                                                                                        | 0/27000 [00:00<?, ?it/s]C:\Users\hp\AppData\Local\Temp\ipykernel_9272\4083466619.py:14: FutureWarning: Possible nested set at position 39
  text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
100%|███████████████████████████████████████████████████████████████████████████| 27000/27000 [00:27<00:00, 976.40it/s]


In [11]:
news['title'].iloc[:10]

0    заместитель председателя правительства рф серг...
1    матч  финала кубка россии по футболу был приос...
2    форвард авангарда томаш заборский прокомментир...
3    главный тренер кубани юрий красножан прокоммен...
4    решением попечительского совета владивостокско...
5    ио главного тренера вячеслав буцаев прокоммент...
6    запорожский металлург дома потерпел разгромное...
7    сборная сша одержала победу над австрией со сч...
8    бывший защитник сборной россии дарюс каспарайт...
9    полузащитник цска зоран тошич после победы над...
Name: title, dtype: object

In [12]:
# Запускаем лемматизацию текста. Будет очень долго...
news['title'] = news['title'].progress_apply(lambda x: lemmatization(x))

100%|███████████████████████████████████████████████████████████████████████████| 27000/27000 [03:15<00:00, 138.15it/s]


А теперь в 3 строчки обучим нашу модель

In [13]:
# сформируем список наших текстов
texts = list(news['title'].values)

# Создадим корпус из списка с текстами
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

Запускаем обучение

In [40]:
N_topic = 22  # можно было менять

In [41]:
from gensim.models import LdaModel

# Обучаем модель на корпусе
lda = LdaModel(common_corpus, num_topics=N_topic, id2word=common_dictionary, passes=2)  # можно было менять

In [43]:
from gensim.test.utils import datapath

# Сохраняем модель на диск
temp_file = datapath("model.lda")
lda.save(temp_file)

In [44]:
# Загружаем обученную модель с диска
lda = LdaModel.load(temp_file)

In [45]:
# Создаем новый корпус документов, которые раньше не видели
other_texts = list(news['title'].iloc[:3])
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'свой', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'мы', 'провести', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'что', 'первый', 'же', 'минута', 'включиться', 'работа', 'сказать', 'заборский', 'мы', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'мы', 'мочь', 'играть', 'ещё', 'хороший', 'но', 'другой', 'сторона', 'пять', 'очко', 'на', 'выезд', 'из', 'девять', 'это', 'хороший', 'чем', 'ничего']


[(2, 0.09109529),
 (5, 0.062397704),
 (7, 0.65308106),
 (14, 0.014821818),
 (15, 0.057945013),
 (16, 0.035917558),
 (17, 0.030244092),
 (18, 0.043053724)]

Обучили модель. Теперь 2 вопроса:

1. как выглядят наши темы
2. как получить для документа вектор значений (вероятности принадлежности каждой теме)

In [46]:
x = lda.show_topics(num_topics=N_topic, num_words=7, formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

# Печатаем только слова
for topic, words in topics_words:
    print(f"topic_{topic}: " + " ".join(words))

topic_0: по закон документ дело орган на он
topic_1: диск ti роджер куст швейцарец бирка дилан
topic_2: что он не на президент украина это
topic_3: население квартира устойчивый сближение антонов киргизия консультация
topic_4: смерть терминал азербайджан художественный лестница вакансия баку
topic_5: на по не это быть что для
topic_6: век авария медведев фотография прибытие приток спектакль
topic_7: на быть день мы планета до не
topic_8: на что не он быть они год
topic_9: год на млн за фонд составить население
topic_10: казахстан эндрю кот пен кейт андерсон казахстанский
topic_11: газ топливо железный сооружение метан проба венгрия
topic_12: на из что быть который он человек
topic_13: напомнить превысить лётчик остров лодка дания болгария
topic_14: британский великобритания лондон рекомендовать мышь сенатор греция
topic_15: на что сша по год быть россия
topic_16: эксперимент налог вирус виза плата студент налоговый
topic_17: город на из по москва быть мероприятие
topic_18: польша польс

Очень неплохо - большинство тем вполне можно описать о чем они

Давайте напишем функцию, которая будет нам возвращать векторное представление новости

In [47]:
def get_lda_vector(lda, text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]

    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(N_topic):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [48]:
get_lda_vector(lda, news['title'].iloc[0])

array([0.12451016, 0.        , 0.76587743, 0.        , 0.        ,
       0.        , 0.        , 0.09543411, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        ])

In [49]:
topic_matrix = pd.DataFrame([get_lda_vector(lda, text) for text in news['title'].values])
topic_matrix.columns = [f'topic_{i}' for i in range(N_topic)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+[f'topic_{i}' for i in range(N_topic)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21
0,6,0.124347,0.0,0.763397,0.0,0.0,0.000000,0.0,0.104665,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
1,4896,0.000000,0.0,0.000000,0.0,0.0,0.331912,0.0,0.380906,0.000000,...,0.137452,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.131486,0.0
2,4897,0.000000,0.0,0.090135,0.0,0.0,0.062360,0.0,0.653579,0.000000,...,0.000000,0.0,0.014743,0.058517,0.035944,0.030237,0.043042,0.0,0.000000,0.0
3,4898,0.000000,0.0,0.091765,0.0,0.0,0.000000,0.0,0.399057,0.475682,...,0.000000,0.0,0.000000,0.011001,0.000000,0.000000,0.000000,0.0,0.000000,0.0
4,4899,0.000000,0.0,0.671528,0.0,0.0,0.000000,0.0,0.108516,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.195895,0.000000,0.0,0.000000,0.0


Прекрасно, мы получили вектора наших новостей! И даже умеем интерпретировать получившиеся темы.

Можно двигаться далее

### Следующий шаг - векторные представления пользователей

In [50]:
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [51]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[[f'topic_{i}' for i in range(N_topic)]].values))

In [52]:
doc_dict[293672]

array([0.        , 0.        , 0.        , 0.        , 0.03411254,
       0.29455543, 0.        , 0.        , 0.17978694, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.10928116, 0.        , 0.        ,
       0.36937544, 0.        ])

In [53]:
def get_user_embedding(user_articles_list, doc_dict, func):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = func(user_vector, axis=0)
    return user_vector

In [54]:
user_articles_list = users['articles'].iloc[33]

get_user_embedding(user_articles_list, doc_dict, np.mean)

array([0.05215769, 0.        , 0.26712954, 0.0051195 , 0.        ,
       0.18834986, 0.        , 0.        , 0.13323614, 0.04593257,
       0.        , 0.        , 0.01012588, 0.00221622, 0.        ,
       0.17561672, 0.        , 0.        , 0.00266475, 0.        ,
       0.10876047, 0.        ])

Теперь получим эмбединги для всех пользователей и проверим их качество на конкретной downstream-задаче

In [55]:
FUNC = np.mean

In [56]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, doc_dict, FUNC))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21
0,u105138,0.060732,0.0,0.138095,0.020019,0.005685,0.075016,0.004219,0.025826,0.240125,...,0.095950,0.000000,0.00000,0.015102,0.00175,0.093414,0.018475,0.000000,0.188521,0.000000
1,u108690,0.052139,0.0,0.278881,0.000000,0.000000,0.138775,0.000000,0.015202,0.230054,...,0.024097,0.000000,0.00000,0.110757,0.00390,0.018705,0.000000,0.004396,0.097919,0.000000
2,u108339,0.051031,0.0,0.178987,0.003901,0.000000,0.101903,0.000000,0.013148,0.096875,...,0.127436,0.002044,0.00486,0.108222,0.00000,0.064116,0.000000,0.001922,0.224392,0.001814


Датасет готов - можно попробовать обучить модель. Загрузим нашу разметку

!wget 'https://drive.google.com/uc?export=download&id=1nWKteQgEr9Rl8CwTRY7N2f7igevNH7oK' -O users_churn.csv

In [57]:
target = pd.read_csv("users_churn.csv")
target.head(3)

,uid,churn
0,u107120,0
1,u102277,0
2,u102444,0


In [58]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,churn
0,u105138,0.060732,0.0,0.138095,0.020019,0.005685,0.075016,0.004219,0.025826,0.240125,...,0.000000,0.00000,0.015102,0.00175,0.093414,0.018475,0.000000,0.188521,0.000000,0
1,u108690,0.052139,0.0,0.278881,0.000000,0.000000,0.138775,0.000000,0.015202,0.230054,...,0.000000,0.00000,0.110757,0.00390,0.018705,0.000000,0.004396,0.097919,0.000000,1
2,u108339,0.051031,0.0,0.178987,0.003901,0.000000,0.101903,0.000000,0.013148,0.096875,...,0.002044,0.00486,0.108222,0.00000,0.064116,0.000000,0.001922,0.224392,0.001814,1


In [59]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

In [60]:
# разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]], 
                                                    X['churn'], random_state=0)

In [61]:
model = LogisticRegression()
# обучим 
model.fit(X_train, y_train)

LogisticRegression()

In [62]:
# наши прогнозы для тестовой выборки
preds = model.predict_proba(X_test)[:, 1]
preds[:10]

array([0.09116069, 0.06513956, 0.51983189, 0.36049827, 0.03299808,
       0.0308552 , 0.02168685, 0.04405592, 0.05573858, 0.02817839])

In [63]:
metrics_df = pd.DataFrame(columns=['model', 'thresh', 'F-Score', 'Precision', 'Recall', 'ROC AUC'])
metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC


In [64]:
from sklearn.metrics import (f1_score, roc_auc_score, precision_score,
                             classification_report, precision_recall_curve, confusion_matrix)

In [65]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')                                                                        

Best Threshold=0.3180702692649966, F-Score=0.700, Precision=0.715, Recall=0.686


In [66]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.9545787545787546

In [67]:
metrics_df = metrics_df.append({
    'model': FUNC.__name__,
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

metrics_df

C:\Users\hp\AppData\Local\Temp\ipykernel_9272\1288159249.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.31807,0.7,0.714894,0.685714,0.954579


2. Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог)

In [68]:
FUNC = np.median

In [69]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, doc_dict, FUNC))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21
0,u105138,0.016811,0.0,0.109483,0.015384,0.0,0.036849,0.0,0.000000,0.190308,...,0.075476,0.0,0.0,0.000000,0.0,0.087686,0.0,0.0,0.161495,0.0
1,u108690,0.035345,0.0,0.283633,0.000000,0.0,0.128034,0.0,0.006519,0.279611,...,0.006692,0.0,0.0,0.057620,0.0,0.007171,0.0,0.0,0.011090,0.0
2,u108339,0.042026,0.0,0.158570,0.000000,0.0,0.094037,0.0,0.000000,0.069080,...,0.119672,0.0,0.0,0.036448,0.0,0.052521,0.0,0.0,0.260571,0.0


In [70]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,churn
0,u105138,0.016811,0.0,0.109483,0.015384,0.0,0.036849,0.0,0.000000,0.190308,...,0.0,0.0,0.000000,0.0,0.087686,0.0,0.0,0.161495,0.0,0
1,u108690,0.035345,0.0,0.283633,0.000000,0.0,0.128034,0.0,0.006519,0.279611,...,0.0,0.0,0.057620,0.0,0.007171,0.0,0.0,0.011090,0.0,1
2,u108339,0.042026,0.0,0.158570,0.000000,0.0,0.094037,0.0,0.000000,0.069080,...,0.0,0.0,0.036448,0.0,0.052521,0.0,0.0,0.260571,0.0,1


In [71]:
# разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]], 
                                                    X['churn'], random_state=0)

In [72]:
model = LogisticRegression()
# обучим 
model.fit(X_train, y_train)

LogisticRegression()

In [73]:
# наши прогнозы для тестовой выборки
preds = model.predict_proba(X_test)[:, 1]
preds[:10]

array([0.19145924, 0.06296851, 0.74802366, 0.44484919, 0.04485231,
       0.03539079, 0.02717207, 0.01232056, 0.32180563, 0.0868649 ])

In [74]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')                                                                        

Best Threshold=0.3282849648851022, F-Score=0.692, Precision=0.711, Recall=0.673


In [75]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.955843944415373

In [76]:
metrics_df = metrics_df.append({
    'model': FUNC.__name__,
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

metrics_df

C:\Users\hp\AppData\Local\Temp\ipykernel_9272\1288159249.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.31807,0.7,0.714894,0.685714,0.954579
1,median,0.328285,0.691824,0.711207,0.673469,0.955844



3. Повторить п.2, но используя уже не медиану, а max


In [77]:
FUNC = np.max

In [78]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, doc_dict, FUNC))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21
0,u105138,0.257817,0.0,0.374679,0.063680,0.034113,0.294555,0.025315,0.154958,0.545562,...,0.212628,0.000000,0.000000,0.090611,0.010501,0.228511,0.110852,0.000000,0.369375,0.000000
1,u108690,0.179558,0.0,0.479393,0.000000,0.000000,0.295930,0.000000,0.062370,0.375684,...,0.076762,0.000000,0.000000,0.366970,0.023402,0.053010,0.000000,0.026378,0.335284,0.000000
2,u108339,0.102132,0.0,0.465805,0.012365,0.000000,0.223976,0.000000,0.078889,0.199257,...,0.294981,0.012264,0.029158,0.332145,0.000000,0.170552,0.000000,0.011534,0.309567,0.010887


In [79]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,churn
0,u105138,0.257817,0.0,0.374679,0.063680,0.034113,0.294555,0.025315,0.154958,0.545562,...,0.000000,0.000000,0.090611,0.010501,0.228511,0.110852,0.000000,0.369375,0.000000,0
1,u108690,0.179558,0.0,0.479393,0.000000,0.000000,0.295930,0.000000,0.062370,0.375684,...,0.000000,0.000000,0.366970,0.023402,0.053010,0.000000,0.026378,0.335284,0.000000,1
2,u108339,0.102132,0.0,0.465805,0.012365,0.000000,0.223976,0.000000,0.078889,0.199257,...,0.012264,0.029158,0.332145,0.000000,0.170552,0.000000,0.011534,0.309567,0.010887,1


In [80]:
# разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]], 
                                                    X['churn'], random_state=0)

In [81]:
model = LogisticRegression()
# обучим 
model.fit(X_train, y_train)

LogisticRegression()

In [82]:
# наши прогнозы для тестовой выборки
preds = model.predict_proba(X_test)[:, 1]
preds[:10]

array([0.0189492 , 0.00561375, 0.54064222, 0.23548404, 0.08233435,
       0.01700561, 0.00582508, 0.14389467, 0.01837701, 0.06218428])

In [83]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')                                                                        

Best Threshold=0.29867092006864904, F-Score=0.674, Precision=0.623, Recall=0.735


In [84]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.9418338275481133

In [85]:
metrics_df = metrics_df.append({
    'model': FUNC.__name__,
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

metrics_df

C:\Users\hp\AppData\Local\Temp\ipykernel_9272\1288159249.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.31807,0.7,0.714894,0.685714,0.954579
1,median,0.328285,0.691824,0.711207,0.673469,0.955844
2,amax,0.298671,0.674157,0.622837,0.734694,0.941834


4. *Воспользовавшись полученными знаниями из п.1, повторить пункт 2, но уже взвешивая новости по tfidf (взяв список новостей пользователя)
	- подсказка 1: нужно получить веса-коэффициенты для каждого документа. Не все документы одинаково информативны и несут какой-то положительный сигнал
	- подсказка 2: нужен именно idf, как вес.

In [86]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [87]:
users['articles_str'] = users['articles'].apply(lambda x: x.replace('[','').replace(']', '').replace(',', ''))

users['articles_str'].iloc[0]

'293672 293328 293001 293622 293126 1852'

In [88]:
tfidf = TfidfVectorizer()
tfidf.fit(users['articles_str'])

TfidfVectorizer()

In [89]:
idf = pd.DataFrame({'article_id': tfidf.get_feature_names_out(),
                    'idf': tfidf.idf_})

idf

,article_id,idf
0,10,8.888710
1,100,7.907880
2,1000,8.041412
3,1001,8.888710
4,1002,8.888710
...,...,...
14776,995,8.377884
14777,996,8.195562
14778,997,8.601027
14779,998,9.294175


In [90]:
def get_user_embedding_idf(user_articles_list, doc_dict):
    user_articles_list = eval(user_articles_list)
    
    user_vector = np.zeros((len(user_articles_list), N_topic))
    for i, doc_id in enumerate(user_articles_list):
        try:
            weight = idf[idf['article_id'] == str(doc_id)]['idf'].values[0]
        except Exception as e:
            weight = 0
        user_vector[i] = doc_dict[doc_id] * weight

    user_vector = np.median(user_vector, axis=0)
    return user_vector

In [91]:
from tqdm import tqdm
tqdm.pandas()

user_embeddings = pd.DataFrame([i for i in users['articles'].progress_apply(lambda x: get_user_embedding_idf(x, doc_dict))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(3)

100%|█████████████████████████████████████████████████████████████████████████████| 8000/8000 [00:56<00:00, 140.86it/s]


,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21
0,u105138,0.156242,0.0,0.994490,0.128385,0.0,0.302000,0.0,0.000000,1.636848,...,0.618571,0.0,0.0,0.000000,0.0,0.754189,0.0,0.0,1.372222,0.0
1,u108690,0.301398,0.0,2.413277,0.000000,0.0,1.159751,0.0,0.053427,2.413994,...,0.059488,0.0,0.0,0.505863,0.0,0.066647,0.0,0.0,0.098574,0.0
2,u108339,0.342617,0.0,1.345528,0.000000,0.0,0.838089,0.0,0.000000,0.569903,...,1.022846,0.0,0.0,0.313488,0.0,0.473669,0.0,0.0,2.190184,0.0


In [92]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,churn
0,u105138,0.156242,0.0,0.994490,0.128385,0.0,0.302000,0.0,0.000000,1.636848,...,0.0,0.0,0.000000,0.0,0.754189,0.0,0.0,1.372222,0.0,0
1,u108690,0.301398,0.0,2.413277,0.000000,0.0,1.159751,0.0,0.053427,2.413994,...,0.0,0.0,0.505863,0.0,0.066647,0.0,0.0,0.098574,0.0,1
2,u108339,0.342617,0.0,1.345528,0.000000,0.0,0.838089,0.0,0.000000,0.569903,...,0.0,0.0,0.313488,0.0,0.473669,0.0,0.0,2.190184,0.0,1


In [93]:
# разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]], 
                                                    X['churn'], random_state=0)

In [94]:
model = LogisticRegression()
# обучим 
model.fit(X_train, y_train)

LogisticRegression()

In [95]:
# наши прогнозы для тестовой выборки
preds = model.predict_proba(X_test)[:, 1]
preds[:10]

array([1.37628084e-01, 2.19289013e-03, 9.53336411e-01, 5.62478544e-01,
       2.04428281e-02, 9.37234027e-03, 5.23666192e-03, 2.78016179e-04,
       4.47987558e-02, 9.74278540e-02])

In [96]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')                                                                        

Best Threshold=0.4878512691110186, F-Score=0.751, Precision=0.817, Recall=0.694


In [97]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.9699912785627072

In [98]:
metrics_df = metrics_df.append({
    'model': 'idf_median',
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

C:\Users\hp\AppData\Local\Temp\ipykernel_9272\1015548053.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({



5. Сформировать на выходе единую таблицу, сравнивающую качество 2/3 разных метода получения эмбедингов пользователей: median, max, idf_mean по метрикам roc_auc, precision, recall, f_score


In [99]:
metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.31807,0.7,0.714894,0.685714,0.954579
1,median,0.328285,0.691824,0.711207,0.673469,0.955844
2,amax,0.298671,0.674157,0.622837,0.734694,0.941834
3,idf_median,0.487851,0.750552,0.817308,0.693878,0.969991
